[View in Colaboratory](https://colab.research.google.com/github/mashaiu/Leaflet.NavBar/blob/master/demo1.ipynb)

In [0]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
#sklearn.metrics模块包括评分函数，性能指标和成对指标以及距离计算。
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
#为将要被记录的的东西（日志）设置开始入口
##日志级别设置成 ERROR，避免干扰
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format
#Python中用format函数格式化字符串的用法
#.1表示长度为1的精度，f表示float类型。

#加载数据集
california_housing_dataframe = pd.read_csv("https://download.mlcc.google.cn/mledu-datasets/california_housing_train.csv", sep=",")
#pandas.read_csv可以读取CSV（逗号分割）文件、文本类型的文件text、log类型到DataFrame, sep设置分隔符

#对数据进行随机化处理
california_housing_dataframe = california_housing_dataframe.reindex(
    np.random.permutation(california_housing_dataframe.index))
california_housing_dataframe["median_house_value"] /= 1000.0
california_housing_dataframe

#会输出关于各列的一些实用统计信息快速摘要：样本数、均值、标准偏差、最大值、最小值和各种分位数。
california_housing_dataframe.describe()

#尝试预测 median_house_value，它将是我们的标签（有时也称为目标）。我们将使用 total_rooms 作为输入特征。
#使用 TensorFlow Estimator API 提供的 LinearRegressor 接口。此 API 负责处理大量低级别模型搭建工作，并会提供执行模型训练、评估和推理的便利方法。

#第 1 步：定义特征并配置特征列
# Define the input feature: total_rooms.
my_feature = california_housing_dataframe[["total_rooms"]]
# Configure a numeric feature column for total_rooms.
feature_columns = [tf.feature_column.numeric_column("total_rooms")]
#从 california_housing_dataframe 中提取 total_rooms 数据，并使用 numeric_column 定义特征列

# 第 2 步：定义标签（目标）
targets = california_housing_dataframe["median_house_value"]

#第 3 步：配置 LinearRegressor
#使用 LinearRegressor 配置线性回归模型，并使用 GradientDescentOptimizer（它会实现小批量随机梯度下降法 (SGD)）训练该模型。learning_rate 参数可控制梯度步长的大小。
#通过 clip_gradients_by_norm 将梯度裁剪应用到我们的优化器。梯度裁剪可确保梯度大小在训练期间不会变得过大，梯度过大会导致梯度下降法失败。
# Use gradient descent as the optimizer for training the model.
my_optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.0000001)
my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
# Configure the linear regression model with our feature columns and optimizer.
# Set a learning rate of 0.0000001 for Gradient Descent.
linear_regressor = tf.estimator.LinearRegressor(
    feature_columns=feature_columns,
    optimizer=my_optimizer
)